In [91]:
import pandas as pd
import pickle
import os
import path

In [92]:
data_files = os.listdir('data')
data_files = [f for f in data_files if f.endswith('.pkl')]

with open('data/' + data_files[2], 'rb') as f:
    data1 = pickle.load(f)

with open('data/' + data_files[3], 'rb') as f:
    data2 = pickle.load(f)

data = data1 + data2

# remove '' and '\n' from each list element
data = [list(
        filter(
            lambda x: 
            x != '' and 
            x != '\n' and 
            x != ' ' and 
            x != ' \n' and 
            x != '.' and
            x != 'Show menu' and
            x != 'Image', d)) for d in data]
data = [d for d in data if len(d) > 0]
print(len(data))
data[0]

3974


['Roberto Marsicano ',
 '@cannedcat.bsky.social',
 '2024/09/21 13:32:48\n1h',
 'A top-1% fund manager warns Warren Buffett is behaving like he did before the dot-com crash — and says the S&P 500 could see dismal returns for a decade\n\n']

In [93]:
# now we have two classes:
# the ones with len = 4 and the rest
index4 = [i for i, d in enumerate(data) if len(d) == 4]
data4 = [d for d in data if len(d) == 4] # name, user, date, text
print(len(data4))
data4[0]

2121


['Roberto Marsicano ',
 '@cannedcat.bsky.social',
 '2024/09/21 13:32:48\n1h',
 'A top-1% fund manager warns Warren Buffett is behaving like he did before the dot-com crash — and says the S&P 500 could see dismal returns for a decade\n\n']

In [94]:
dataRest = [d for d in data if len(d) != 4]
print(len(dataRest))

1853


In [95]:
index5 = [i for i, d in enumerate(dataRest) if len(d) == 5]
data5 = [d for d in dataRest if len(d) == 5]
# removing last element from each list
data5 = [d[:-1] for d in data5]
print(len(data5))
data5[0]

674


['@ilainme.bsky.social',
 '@IlainME  Voting for Freedom: Harris/Walz 2024 Not Going Back',
 '2024/09/20 15:51:37\n22h',
 'S&P 500, Dow notch record highs after Fed rate cut\n\n']

In [96]:
new_index4 = []
new_data4 = data4 + data5
print(len(new_data4))
new_data4[0]

2795


['Roberto Marsicano ',
 '@cannedcat.bsky.social',
 '2024/09/21 13:32:48\n1h',
 'A top-1% fund manager warns Warren Buffett is behaving like he did before the dot-com crash — and says the S&P 500 could see dismal returns for a decade\n\n']

In [97]:
dataRest = [d for d in dataRest if len(d) != 5]
print(len(dataRest))

print(len(dataRest) + len(new_data4) == len(data))

1179
True


In [98]:
data3 = [d for d in dataRest if len(d) == 3]
# im guessing: user, date, text
print(len(data3)) 
# assert first element of each starts with '@'
# if not, drop it
new_data3 = [d for d in data3 if d[0].startswith('@')]
print(len(new_data3))
data3[0]

138
81


['@beadoz.bsky.social',
 '2024/09/20 20:10:38\n18h',
 'The S&P 500 will plunge 32% in 2025 as a recession finally hits the US economy, BCA Research predicts. The firm said the Fed will fail to prevent a recession as it takes its time cutting interest rates. Rising unemployment and constrained credit will curb consumer spending, worsening the downturn.']

In [99]:
dataRest = [d for d in dataRest if len(d) != 3]
print(len(dataRest))

1041


In [100]:
# prepend 'null' to each list element in data3
new_data3_ = [['null'] + d for d in new_data3]
print(len(new_data3_))
new_data3_[0]

81


['null',
 '@beadoz.bsky.social',
 '2024/09/20 20:10:38\n18h',
 'The S&P 500 will plunge 32% in 2025 as a recession finally hits the US economy, BCA Research predicts. The firm said the Fed will fail to prevent a recession as it takes its time cutting interest rates. Rising unemployment and constrained credit will curb consumer spending, worsening the downturn.']

In [101]:
new_data = new_data4 + new_data3_

In [161]:
import tldextract
import re
import emoji
import unicodedata

def treat_text(text):
    words = text.split()
    # remove '(' and ')' from each word
    words = [w.replace('(', '').replace(')', '') for w in words]
    cleaned_words = []
    for word in words:
        ext = tldextract.extract(word)
        if ext.domain and ext.suffix:
            # Detected as a URL, skip it
            continue
        cleaned_words.append(word)
    # drop single word texts
    if len(words) == 1:
        return ''
    return ' '.join(cleaned_words)

def normalize_text(text):
    # Remover acentos e caracteres especiais
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    # Remover emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Símbolos e pictogramas
        u"\U0001F680-\U0001F6FF"  # Transportes e símbolos de mapa
        u"\U0001F1E0-\U0001F1FF"  # Bandeiras (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    # Converter para letras minúsculas
    text = text.lower()
    return text

In [166]:
df = pd.DataFrame(new_data, columns=['name', 'user', 'date', 'text'])

In [167]:
# drop date values that are not numeric
df = df[df['date'].apply(lambda x: x.split()[0][0].isnumeric())]

# drop user values that are not starting with '@'
df = df[df['user'].apply(lambda x: x.startswith('@'))]

# remove urls and '()' from each text
df['text'] = df['text'].apply(treat_text)

# remove empty texts
df = df[df['text'] != '']

In [168]:
from langdetect import detect

def is_not_english(text):
    try:
        return detect(text) != 'en'
    except:
        return True
    
df['is_not_english'] = df['text'].apply(is_not_english)

In [171]:
from googletrans import Translator
from tqdm import tqdm
import time

def translate_text(text, retries=3, delay=5):
    translator = Translator()
    for i in range(retries):
        try:
            return translator.translate(text).text
        except Exception as e:
            if i < retries - 1:
                print(f"Erro: {e}. Tentando novamente em {delay} segundos...")
                time.sleep(delay)
            else:
                print(f"Falha ao traduzir o texto após {retries} tentativas.")
                return text  # Retorna o texto original em caso de falha

# Adicionar uma barra de progresso
tqdm.pandas()

# if df['is_not_english'] is True, translate text to a new column else, keep the original text
df['translated_text'] = df['text']
df.loc[df['is_not_english'], 'translated_text'] = df.loc[df['is_not_english'], 'text'].progress_apply(translate_text)

100%|██████████| 732/732 [11:03<00:00,  1.10it/s]


In [174]:
df['translated_text'] = df['translated_text'].apply(normalize_text)

df = df.drop(columns=['is_not_english'])
df['text'] = df['translated_text']
df = df.drop(columns=['translated_text'])

In [175]:
# split every date value and see unique values
len(df['date'].apply(lambda x: x.split()[0]).unique())

466

In [176]:
len(df)

2550

In [177]:
with open('data/df_snapshot2.pkl', 'wb') as f:
    pickle.dump(df, f)

In [178]:
df[df['user'] == '@financebot.bsky.social'].iloc[0]['text']

'nikkei 225 closing price: 36391.469 yen before hinoto: -0.72% s & p 500: 5503.41 the day from the previous day: -0.3% nasdaq composite: 17127.66 the previous day:+0.25% vix: 19.9 the previous day: -6.66%'

In [183]:
print(df['text'].sample(10).values)

['live: dow tumbles 1,000 points, s&p 500 posts worst day since 2022 in global market sell-off'
 'the s&p 500, which is a better indicator of the health of the economy, is up even more that the dji:'
 'the typical s&p 500 company ceo received an average of $17.7 million in 2023, 268 times more than their median employees pay. a typical worker would have had to begin working before the american revolution to earn what s&p 500 company ceos made on average in just one year. '
 'the s&p 500 is up 25% this year. have patience and a long-term mindset when it comes to investing. long term investing transfers wealth to the patient from the impatient. investing may be risky in the short term, but not investing is riskier in the long term.'
 'the us s & p 500 index market 5,000 lessons...6 consecutive trading days'
 'he just started talking about the s&p 500 lmao'
 's&p 500 mutual fund fees dilemma'
 'nikkei 225 closing price: 31624.279 yen before hikini: -2.05 s & p 500: 4405.71 the previous da

Essa parte para baixo não foi utilizada

In [84]:
[d for d in dataRest if len(d) == 2][0]

['The US stock market S&P 500 today closed at a new record high.\n\nThis is awesome news for the view on Democratic economic policies, and very inconsistent with Trump\'s "America is going to hell" message […]',
 'mastodon.social\nOriginal post on mastodon.social']

In [85]:
import tldextract

def treat_text(text):
    words = text.split()
    # remove '(' and ')' from each word
    words = [w.replace('(', '').replace(')', '') for w in words]
    cleaned_words = []
    for word in words:
        ext = tldextract.extract(word)
        if ext.domain and ext.suffix:
            # Detected as a URL, skip it
            continue
        cleaned_words.append(word)
    # drop single word texts
    if len(words) == 1:
        return ''
    return ' '.join(cleaned_words)


data1 = [d for d in dataRest if len(d) == 1]
data1_cleaned = [treat_text(d[0]) for d in data1]
# im guessing: text
print(len(data1_cleaned))
data1_cleaned[0]

543


'A top-1% fund manager warns Warren Buffett is behaving like he did before the dot-com crash — and says the S&P 500 could see dismal returns for a decade'